<a href="https://colab.research.google.com/github/williserdman/python_playground/blob/main/basic_text_classification/main_tf_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys


In [ ]:
print("System version:", sys.version)
import tensorflow as tf


System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [ ]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds


In [ ]:
print("Tensorflow version: ", tf.__version__)
print("Tensorflow eager mode: ", tf.executing_eagerly())
print("Tensorflow Hub version: ", hub.__version__)
print(
    "GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE"
)
tf.config.list_physical_devices()


Tensorflow version:  2.8.0
Tensorflow eager mode:  True
Tensorflow Hub version:  0.12.0
GPU is available


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
train_ds, val_ds, test_ds = tfds.load(
    name="imdb_reviews",
    split=("train[:60%]", "train[60%:]", "test"),
    as_supervised=True,
)


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_ds.batch(10)))
print(train_examples_batch)
print(train_labels_batch)


tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
example_hub_layer = hub_layer(train_examples_batch[:3])
print(example_hub_layer)


tf.Tensor(
[[ 1.15015066e+00  7.80130103e-02  9.26615447e-02  2.83361435e-01
   9.67164431e-03 -1.49186060e-01  3.35665733e-01 -3.50244790e-01
  -8.28830618e-03 -1.87714081e-02 -3.33069824e-02 -6.33094192e-01
  -3.75421256e-01 -2.77732849e-01 -9.66175571e-02  1.72553629e-01
  -1.33676574e-01  3.80765870e-02 -2.75138199e-01  4.94762301e-01
   3.93051840e-02  1.34496227e-01 -2.70728230e-01  1.78942848e-02
  -2.41071522e-01  2.71089785e-02  1.02333419e-01 -1.06628530e-01
   5.24298586e-02  1.19170472e-01 -6.67077256e-03  3.39231521e-01
   1.13014966e-01  1.06842585e-01  3.91571134e-01 -1.89536288e-01
  -1.74000353e-01 -1.06444150e-01 -1.34200439e-01  1.73583925e-01
  -2.77695060e-01 -4.33591381e-02 -3.91500629e-02 -1.98340893e-01
   2.74854712e-02  2.76702970e-01  1.40702844e-01 -3.14256102e-01
   2.47569270e-02 -1.67308271e-01  8.21740180e-02 -9.56841931e-02
  -2.57335782e-01  3.72914791e-01 -2.21059948e-01 -2.14436904e-01
  -5.83256371e-02  1.50036931e-01 -1.67723849e-01 -8.02809969e-02

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))


In [ ]:
model.summary()
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_2 (Dense)             (None, 16)                2064      
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#this is c
history = model.fit(
    train_ds.shuffle(10000).batch(512),
    epochs=10,
    validation_data=val_ds.batch(512),
    verbose=1
)


Epoch 1/10
30/30 [==============================] - 47s 2s/step - loss: 0.6034 - accuracy: 0.5911 - val_loss: 0.5015 - val_accuracy: 0.7523
Epoch 2/10
30/30 [==============================] - 45s 1s/step - loss: 0.3937 - accuracy: 0.8346 - val_loss: 0.3548 - val_accuracy: 0.8551
Epoch 3/10
30/30 [==============================] - 45s 1s/step - loss: 0.2481 - accuracy: 0.9095 - val_loss: 0.2937 - val_accuracy: 0.8658
Epoch 4/10
30/30 [==============================] - 46s 2s/step - loss: 0.1617 - accuracy: 0.9445 - val_loss: 0.2692 - val_accuracy: 0.8879
Epoch 5/10
30/30 [==============================] - 45s 1s/step - loss: 0.1045 - accuracy: 0.9700 - val_loss: 0.2648 - val_accuracy: 0.8908
Epoch 6/10
30/30 [==============================] - 45s 2s/step - loss: 0.0659 - accuracy: 0.9852 - val_loss: 0.2718 - val_accuracy: 0.8904
Epoch 7/10
30/30 [==============================] - 45s 1s/step - loss: 0.0414 - accuracy: 0.9926 - val_loss: 0.2864 - val_accuracy: 0.8875
Epoch 8/10
30/30 [==

In [24]:
results = model.evaluate(test_ds.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3471 - accuracy: 0.8697 - 3s/epoch - 65ms/step
loss: 0.347
accuracy: 0.870
